In [2]:
from dateutil.relativedelta import relativedelta
from collections import namedtuple
from datetime import datetime
from dotenv import load_dotenv

import unicodedata
import firebirdsql
import time
import os

In [3]:
def remove_accents(input_str):
    # Função auxiliar para remover acentos (caso já não esteja definida no seu código)
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])


def get_firebird_connection():
    # Ajuste com os parâmetros corretos do Firebird, inclusive charset
    return firebirdsql.connect(
        host=os.getenv('HOST'),
        port=os.getenv('PORT'),
        database=os.getenv('DB_PATH'),
        user=os.getenv('USER'),
        password=os.getenv('PASSWORD'),
        role=os.getenv('ROLE'),
        auth_plugin_name=os.getenv('AUTH'),
        wire_crypt=False,
        charset='ISO8859_1'
    )

In [8]:
try:
    conn = firebirdsql.connect(
        host='100.64.1.10', 
        port=3050,
        database='C:\\Micromais\\mmSeculos\\BD\\SECULOS.MMDB',
        user='USER_CONSULTA',
        password='consultaseculos',
        role='apenasconsulta',
        auth_plugin_name='Legacy_Auth',
        wire_crypt=False,
        charset='ISO8859_1'
    )

    print("Conexão bem-sucedida ao banco de dados!")
    cursor = conn.cursor()
    
    returned_orders_query = '''
        SELECT 
            p.DATA AS DataPedido,
            SUM(d.VALORTOTAL) AS TotalDevolvido
        FROM 
            PEDIDOVENDA p
        INNER JOIN 
            DEVOLUCAO d
        ON 
            p.CDPEDIDOVENDA = d.CDPEDIDOVENDA
        WHERE 
            p.DATA BETWEEN '2025-05-01' AND CURRENT_DATE - 1
            AND p.DEVOLVIDO = 'S'
        GROUP BY 
            p.DATA
        ORDER BY 
            p.DATA;
    '''
    
    cursor.execute(returned_orders_query)
    returned_orders_results = cursor.fetchall()
    
    display(returned_orders_results)
    soma = 0
    for row in returned_orders_results:
        soma += row[1]

    print(f"Soma total dos valores devolvidos: {soma}")

except firebirdsql.Error as e:
    print(f"Erro ao conectar ao banco de dados: {e}")


Conexão bem-sucedida ao banco de dados!


[(datetime.date(2025, 5, 5), Decimal('748.70')),
 (datetime.date(2025, 5, 6), Decimal('1237.48')),
 (datetime.date(2025, 5, 7), Decimal('1470.43'))]

Soma total dos valores devolvidos: 3456.61


In [ ]:
from datetime import datetime
import firebirdsql
import pandas as pd

def get_sales_data():
    try:
        # Conexão com o banco de dados Firebird
        conn = firebirdsql.connect(
            host='100.64.1.10', 
            port=3050,
            database='C:\\Micromais\\mmSeculos\\BD\\SECULOS.MMDB',
            user='USER_CONSULTA',
            password='consultaseculos',
            role='apenasconsulta',
            auth_plugin_name='Legacy_Auth',
            wire_crypt=False,
            charset='ISO8859_1'  # Charset ajustado
        )
        
        print("Conexão bem-sucedida ao banco de dados!")
        
        cursor = conn.cursor()
        
        # Total diário de vendas para Jucilande e Josuilton
        total_query = '''
            SELECT 
                p.DATA,
                f.NOME,
                SUM(p.VALORCDESC) AS TOTAL_VENDAS
            FROM 
                PEDIDOVENDA p
            INNER JOIN 
                FUNCIONARIO f
            ON 
                p.CDFUNC = f.CDFUNC
            INNER JOIN 
                CLIENTE c
            ON 
                p.CDCLIENTE = c.CDCLIENTE
            WHERE 
                p.EFETIVADO = 'S'
                AND p.DATA BETWEEN '2024-12-01' AND CURRENT_DATE
                AND UPPER(c.NOME) NOT LIKE '%COMAGRO%'
            GROUP BY 
                p.DATA, f.NOME
            ORDER BY 
                p.DATA, f.NOME;
        '''
        
        cursor.execute(total_query)
        total_results = cursor.fetchall()
        
        display("Total de vendas diárias:", total_results)
        
        # Criar DataFrame para o total diário
        total_df = pd.DataFrame(total_results, columns=["Data", "Vendedor", "Total Vendas"])
        
        # Média diária de vendas por vendedor
        avg_query = '''
            SELECT 
                DIARIO.NOME,
                AVG(DIARIO.TOTAL_VENDAS) AS MEDIA_DIARIA
            FROM (
                SELECT 
                    p.DATA,
                    f.NOME AS NOME,
                    SUM(p.VALORCDESC) AS TOTAL_VENDAS
                FROM 
                    PEDIDOVENDA p
                INNER JOIN 
                    FUNCIONARIO f
                ON 
                    p.CDFUNC = f.CDFUNC
                INNER JOIN 
                    CLIENTE c
                ON 
                    p.CDCLIENTE = c.CDCLIENTE
                WHERE 
                    p.EFETIVADO = 'S'
                    AND p.DATA BETWEEN '2024-12-01' AND CURRENT_DATE
                    AND UPPER(c.NOME) NOT LIKE '%COMAGRO%'
                GROUP BY 
                    p.DATA, f.NOME
            ) DIARIO
            GROUP BY 
                DIARIO.NOME
            ORDER BY 
                DIARIO.NOME;
        '''
        
        cursor.execute(avg_query)
        avg_results = cursor.fetchall()
        
        display("Média diária de vendas por vendedor:", avg_results)
        
        # Criar DataFrame para a média diária
        avg_df = pd.DataFrame(avg_results, columns=["Vendedor", "Média Diária"])

        # Convertendo valores para float
        total_df["Total Vendas"] = total_df["Total Vendas"].astype(float)
        avg_df["Média Diária"] = avg_df["Média Diária"].astype(float)
        
        # Salvar resultados em uma planilha Excel
        with pd.ExcelWriter("arquivos/vendas_vendedores.xlsx") as writer:
            total_df.to_excel(writer, sheet_name="Total Diário", index=False)
            avg_df.to_excel(writer, sheet_name="Média Diária", index=False)
        
        print("Planilha Excel criada com sucesso: vendas_vendedores.xlsx")
        conn.close()
                
    except firebirdsql.Error as e:
        print(f"Erro ao conectar ao banco de dados: {e}")


# Executa o código
get_sales_data()

In [ ]:
from datetime import datetime
import firebirdsql
import pandas as pd

def get_best_day_sales():
    try:
        conn = firebirdsql.connect(
            host='100.64.1.10', 
            port=3050,
            database='C:\\Micromais\\mmSeculos\\BD\\SECULOS.MMDB',
            user='USER_CONSULTA',
            password='consultaseculos',
            role='apenasconsulta',
            auth_plugin_name='Legacy_Auth',
            wire_crypt=False,
            charset='ISO8859_1'
        )

        print("Conexão bem-sucedida ao banco de dados!")
        cursor = conn.cursor()
        
        best_day_query = '''
            SELECT 
                ds.vendedor, 
                ds.dia, 
                ds.total_dia
            FROM (
                SELECT 
                    f.NOME AS vendedor,
                    p.DATA AS dia,
                    SUM(p.VALORCDESC) AS total_dia
                FROM 
                    PEDIDOVENDA p
                    INNER JOIN FUNCIONARIO f ON p.CDFUNC = f.CDFUNC
                    INNER JOIN USUARIO u ON f.CDFUNC = u.CDFUNC
                    INNER JOIN CLIENTE c ON p.CDCLIENTE = c.CDCLIENTE
                WHERE 
                    p.EFETIVADO = 'S'
                    AND p.DATA BETWEEN '2000-01-01' AND CURRENT_DATE
                    AND UPPER(c.NOME) NOT LIKE '%COMAGRO%'
                    AND u.HABILITADO = 'S'
                GROUP BY 
                    f.NOME, 
                    p.DATA
            ) ds
            WHERE ds.total_dia = (
                SELECT 
                    MAX(ds2.total_dia)
                FROM (
                    SELECT 
                        f.NOME AS vendedor,
                        p.DATA AS dia,
                        SUM(p.VALORCDESC) AS total_dia
                    FROM 
                        PEDIDOVENDA p
                        INNER JOIN FUNCIONARIO f ON p.CDFUNC = f.CDFUNC
                        INNER JOIN USUARIO u ON f.CDFUNC = u.CDFUNC
                        INNER JOIN CLIENTE c ON p.CDCLIENTE = c.CDCLIENTE
                    WHERE 
                        p.EFETIVADO = 'S'
                        AND p.DATA BETWEEN '2000-01-01' AND CURRENT_DATE
                        AND UPPER(c.NOME) NOT LIKE '%COMAGRO%'
                        AND u.HABILITADO = 'S'
                    GROUP BY 
                        f.NOME, 
                        p.DATA
                ) ds2
                WHERE ds2.vendedor = ds.vendedor
            )
            ORDER BY ds.vendedor
        '''
        
        cursor.execute(best_day_query)
        best_day_results = cursor.fetchall()
        
        display("Melhor dia de vendas por vendedor habilitado:", best_day_results)
        
        best_day_df = pd.DataFrame(best_day_results, columns=["Vendedor", "Melhor Dia", "Total Vendas no Dia"])
        
        # Salva resultados em Excel
        with pd.ExcelWriter("arquivos/vendas_vendedores.xlsx") as writer:
            best_day_df.to_excel(writer, sheet_name="Melhor Dia", index=False)
        
        print("Planilha Excel criada com sucesso: vendas_vendedores.xlsx")
        conn.close()

    except firebirdsql.Error as e:
        print(f"Erro ao conectar ao banco de dados: {e}")

# Executa
get_best_day_sales()